### 先都加载到cpu上

In [31]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"
os.environ["TOKENIZERS_PARALLELISM"] = "False"
# from modeling_mixtral import MixtralForCausalLM
from transformers import AutoTokenizer, MixtralForCausalLM
import torch
import torch.nn as nn
from typing import Optional
import json

with open('../path.json', 'r') as f:
    path = json.load(f)
    model_name = path['mixtral']
    # threshold_path = path[threshold_path_name]

with open("../quantize/device_map.json", "r") as f:
    device_map = json.load(f)

def get_model(model_name, device_map, dtype=torch.bfloat16, use_cache=True):
    llm = MixtralForCausalLM.from_pretrained(
        model_name,
        device_map=device_map,
        use_cache=use_cache,
        torch_dtype=dtype,
    ) 
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

    return llm, tokenizer

dtype = torch.float16
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
llm, tokenizer = get_model(model_name, 'cpu', dtype=dtype)

Loading checkpoint shards: 100%|██████████| 19/19 [00:07<00:00,  2.67it/s]


In [2]:
### HQQ量化
from hqq.core.quantize import *
from hqq.models.hf.mixtral import MixtralHQQ

# save_dir = './hqqsaved'
### 第一次加载
# q3_config    = BaseQuantizeConfig(nbits=2, group_size=64)
# quant_config      = {'block_sparse_moe.experts.w3'   : q3_config}
# llm = MixtralForCausalLM.from_pretrained(
#         model_name,
#         device_map='cpu',
#         use_cache=True,
#         torch_dtype=dtype,
#     ) 
# MixtralHQQ.quantize_model(llm, quant_config=quant_config, compute_dtype=dtype, device='cuda:0')
#### 先放CUDA量化，然后再传回CPU
# MixtralHQQ.save_quantized(llm, save_dir)

### 从保存的权重中加载
# llm = MixtralHQQ.from_quantized(save_dir, compute_dtype=dtype, device='cpu')
# HQQLinear.set_backend(HQQBackend.PYTORCH)

# backend       = "gemlite" #'torchao_int4' #"torchao_int4" (4-bit only) or "gemlite" (4-bit + 2-bit)
# #Optimize
# from hqq.utils.patching import prepare_for_inference
# prepare_for_inference(llm, backend=backend, verbose=True)
# #Load GemLite cache
# if(backend == 'gemlite'):
# 	import gemlite
# 	gemlite.core.GEMLITE_TRITON_RESTRICT_M = True
# 	gemlite.core.GemLiteLinear.load_config('/tmp/gemlite_config.json')

/home/bcds/venv/dilab/floe/hqq/hqq/models/base.py:251: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(cls.get_weight_file(save_dir), map_location=map_locati

In [32]:
from typing import Tuple, Optional
import torch
import torch.nn as nn
import threading
import json
import torch.nn.functional as F

#### 增加专家preload失败后的重新加载
class CachedMLP(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, dtype, sparsity: float = 0.2):
        super(CachedMLP, self).__init__()
        self.sparsity = sparsity
        self.activenum = int((1 - sparsity) * hidden_dim)
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.dtype = dtype
        print("active neural num ",self.activenum)

        self.activation = nn.SiLU()

        #### 中间变量
        self.w3_result1 = None
        self.w3_result2 = None

        # 将GPU缓存张量改为列表存储
        self.w_gpu = [
            torch.empty((self.activenum, self.input_dim), dtype=self.dtype, device='cuda:0') for _ in range(4)
        ]  # [w1_gpu, w2_gpu, w1_gpu_expert1, w2_gpu_expert1]

        # 将Pinned Memory缓冲区改为列表存储
        self.sparse_w_cpu = [
            torch.empty((self.activenum, self.input_dim), dtype=self.dtype, device='cpu').pin_memory() for _ in range(4)
        ]  # [sparse_w1_cpu, sparse_w2_cpu, sparse_w1_cpu_expert1, sparse_w2_cpu_expert1]

        ### 增加两个专家序号
        self.expert_ids = torch.tensor([0,1])

    def get_predict_experts(self):
        return self.expert_ids
    
    def load_conflict_cpu(self, cpu_mlp_new, stream: torch.cuda.Stream, hidden_states, reuse_idx):
        # 只更新需要加载的专家权重
        _, indices = torch.topk(cpu_mlp_new['w3'](hidden_states), self.activenum, dim=1)
        indices = indices[0].cpu()
        if reuse_idx == 0:
            # 更新CPU数据
            self.sparse_w_cpu[2].copy_(cpu_mlp_new['w1'].data[indices, :])
            self.sparse_w_cpu[3].copy_(cpu_mlp_new['w2'].data[indices, :])
            
            # 异步复制到GPU
            with torch.cuda.stream(stream):
                self.w_gpu[2].copy_(self.sparse_w_cpu[2], non_blocking=True)
                self.w_gpu[3].copy_(self.sparse_w_cpu[3], non_blocking=True)
        else:
            # 更新CPU数据
            self.sparse_w_cpu[0].copy_(cpu_mlp_new['w1'].data[indices, :])
            self.sparse_w_cpu[1].copy_(cpu_mlp_new['w2'].data[indices, :])
            
            # 异步复制到GPU
            with torch.cuda.stream(stream):
                self.w_gpu[0].copy_(self.sparse_w_cpu[0], non_blocking=True)
                self.w_gpu[1].copy_(self.sparse_w_cpu[1], non_blocking=True)

    def load_from_cpu(self, cpu_mlp, cpu_mlp_expert1, stream: torch.cuda.Stream, hidden_states):
        """
        从CPU加载参数，并使用指定的CUDA流进行异步复制到GPU。
        
        参数:
            cpu_mlp: 包含CPU上参数的字典（第一个专家）
            cpu_mlp_expert1: 包含CPU上参数的字典（第二个专家）。
            stream: 用于数据传输的CUDA流。
        """
        ### 根据up计算的结果进行稀疏化
        up_result1 = cpu_mlp['w3'](hidden_states)
        up_result2 = cpu_mlp_expert1['w3'](hidden_states)
        # 提取 up_result1 的值并计算 top-k 索引
        _, indices1 = torch.topk(up_result1, self.activenum, dim=1)  # 在第二个维度上取 top-k
        self.w3_result1 = up_result1[: , indices1[0]]
        indices1 = indices1[0].cpu()

        _, indices2 = torch.topk(up_result2, self.activenum, dim=1)  # 在第二个维度上取 top-k
        self.w3_result2 = up_result2[: , indices2[0]]
        indices2 = indices2[0].cpu() 

        # 使用列表索引更新CPU数据
        self.sparse_w_cpu[0].copy_(cpu_mlp['w1'].data[indices1, :])
        self.sparse_w_cpu[1].copy_(cpu_mlp['w2'].data[indices1, :])
        self.sparse_w_cpu[2].copy_(cpu_mlp_expert1['w1'].data[indices2, :])
        self.sparse_w_cpu[3].copy_(cpu_mlp_expert1['w2'].data[indices2, :])
        
        # 异步复制到GPU
        with torch.cuda.stream(stream):
            self.w_gpu[0].copy_(self.sparse_w_cpu[0], non_blocking=True)
            self.w_gpu[1].copy_(self.sparse_w_cpu[1], non_blocking=True)
            self.w_gpu[2].copy_(self.sparse_w_cpu[2], non_blocking=True)
            self.w_gpu[3].copy_(self.sparse_w_cpu[3], non_blocking=True)

    def load_expert_weights(self, expert_ids):
        # print('loading next expert: ', expert_ids)   ## tensor([7, 6], device='cuda:0')
        self.expert_ids = expert_ids

    def forward(self, hidden_states, expert_weights, expert_ids):
        """
        根据hidden_states， 分别计算两个专家的输出
        """
        ### 如果取出来的专家 第一个对不上，可能是预测的 1，2号专家顺序颠倒了，替换
        if self.expert_ids[0] != expert_ids[0]:
            expert_weights[0], expert_weights[1] = expert_weights[1], expert_weights[0]

        w3_output = self.w3_result1
        w1_output = self.activation(torch.matmul(hidden_states, self.w_gpu[0].T))
        hidden_states_expert0 = torch.matmul(w1_output * w3_output, self.w_gpu[1])

        # 第二个专家的计算
        w3_output_expert1 = self.w3_result2
        w1_output_expert1 = self.activation(torch.matmul(hidden_states, self.w_gpu[2].T))
        hidden_states_expert1 = torch.matmul(w1_output_expert1 * w3_output_expert1, self.w_gpu[3])

        final_hidden_states = hidden_states_expert0 * expert_weights[0] + hidden_states_expert1 * expert_weights[1]
        return final_hidden_states
                    

class PipelineLLM:
    def __init__(self, llm, cached_mlps):
        """
        初始化 PipelineLLM，替换模型每一层的 forward 方法。
        
        参数:
            llm: 原始的大模型
            cached_mlps: 两个 CachedMLP 实例列表
        """
        self.llm = llm
        self.cached_mlps = cached_mlps  # [buffer0, buffer1]
        self.num_layers = len(llm.model.layers)
        self.lock = threading.Lock()
        self.use_buffer0 = True  # 标记当前使用哪个缓冲区

        self.stream0 = torch.cuda.Stream()

        self.top_k = 2
        self.activation = nn.SiLU()

        self._replace_forward_methods()

        # 用于统计时间的变量
        self.total_reload_experts = 0
    
    def get_reload_experts(self):
        return self.total_reload_experts

    def print_reload_experts(self):
        print("all reload experts are:", self.total_reload_experts)
        self.total_reload_experts = 0

    def _load_layer(self, layer_idx, buffer, expert_ids,
                    hidden_states):
        """
        加载指定层的参数到指定的缓冲区。
        
        参数:
            layer_idx: 层的索引
            buffer_index: 缓冲区的索引（0 或 1）
        """
        layer = self.llm.model.layers[layer_idx]
        expert0 = layer.block_sparse_moe.experts[expert_ids[0]]
        expert1 = layer.block_sparse_moe.experts[expert_ids[1]]

        cpu_mlp = expert0.cpu_mlp
        cpu_mlp_expert1 = expert1.cpu_mlp

        ### weights应该用正确的来算
        buffer.load_expert_weights(expert_ids)
        # 异步加载参数
        buffer.load_from_cpu(cpu_mlp, cpu_mlp_expert1, self.stream0, hidden_states)

    def _load_conflict_layer(self, layer_idx, buffer, expert_ids, hidden_states):
        """
        处理专家预测冲突的情况，尽可能复用已加载的专家权重
        
        参数:
            layer_idx: 层索引
            buffer_index: 缓冲区索引
            expert_ids: 实际需要的专家ID
            predict_experts: 预测的专家ID
            hidden_states: 输入hidden_states
        """
        predict_experts = buffer.get_predict_experts()
        print(f"predict_experts in layer {layer_idx}: {predict_experts}")
        # 找出需要加载的新专家
        required_experts = set(expert_ids.tolist()) - set(predict_experts.tolist())
        
        # 如果只需要加载一个专家
        if len(required_experts) == 1:
            new_expert_id = list(required_experts)[0]
            # 找出可以复用的专家位置
            reuse_idx = 0 if expert_ids[0] in predict_experts else 1

            print(f"reloading 1 experts, {expert_ids[1-reuse_idx]}")
            self.total_reload_experts += 1
            
            # 获取需要加载的专家
            layer = self.llm.model.layers[layer_idx]
            new_expert = layer.block_sparse_moe.experts[new_expert_id]
            cpu_mlp_new = new_expert.cpu_mlp
            
            # 只更新需要加载的专家权重
            buffer.load_conflict_cpu(cpu_mlp_new, self.stream0, hidden_states, reuse_idx)
            
            # 更新专家ID
            buffer.load_expert_weights(expert_ids)
        else:
            print(f"reloading 2 experts, {expert_ids[:]}")
            self.total_reload_experts += 2
            # 需要加载两个专家，直接调用原始方法
            self._load_layer(layer_idx, buffer, expert_ids, hidden_states)

    def _replace_forward_methods(self):
        """
        替换模型每一层的 forward 方法，添加参数预加载逻辑和注意力计算。
        """
        for i, layer in enumerate(self.llm.model.layers):
            def new_forward(hidden_states: torch.Tensor,
                        attention_mask: Optional[torch.Tensor] = None,
                        position_ids: Optional[torch.LongTensor] = None,
                        past_key_value: Optional[Tuple[torch.Tensor]] = None,
                        output_attentions: Optional[bool] = False,
                        output_router_logits: Optional[bool] = False,
                        use_cache: Optional[bool] = False,
                        cache_position: Optional[torch.LongTensor] = None,
                        layer=layer,
                        layer_idx=i):
                with self.lock:
                    batch_size, sequence_length, hidden_dim = hidden_states.shape
                    
                    if sequence_length == 1:
                        #### decode phase ####
                        # 选择当前使用的缓冲区
                        current_buffer = self.cached_mlps[0] if self.use_buffer0 else self.cached_mlps[1]
                        
                        next_buffer_index = 1 if self.use_buffer0 else 0
                        next_layer_idx = layer_idx + 1
                        if next_layer_idx < self.num_layers:
                            ### 使用下一个缓冲区进行加载
                            next_buffer = self.cached_mlps[next_buffer_index]

                            # 预加载下一层的参数
                            next_layer = self.llm.model.layers[next_layer_idx]
                            router = next_layer.block_sparse_moe.gate

                            # batch_size, sequence_length, hidden_dim = hidden_states.shape
                            hidden_states_flat = hidden_states.view(-1, hidden_dim)
                            # router_logits: (batch * sequence_length, n_experts)
                            router_logits = router(hidden_states_flat)

                            routing_weights = F.softmax(router_logits, dim=1, dtype=torch.float)
                            routing_weights, selected_experts = torch.topk(routing_weights, self.top_k, dim=-1)
                            routing_weights /= routing_weights.sum(dim=-1, keepdim=True)

                            self._load_layer(
                                next_layer_idx,
                                buffer=next_buffer,
                                expert_ids=selected_experts[0],
                                hidden_states=hidden_states_flat,
                            )
                            hidden_states = hidden_states_flat.reshape(batch_size, sequence_length, hidden_dim)

                            # 切换缓冲区
                            self.use_buffer0 = not self.use_buffer0

                    # 处理当前层
                    residual = hidden_states
                    hidden_states = layer.input_layernorm(hidden_states)

                    # Self Attention
                    hidden_states, self_attn_weights, present_key_value = layer.self_attn(
                        hidden_states=hidden_states,
                        attention_mask=attention_mask,
                        position_ids=position_ids,
                        past_key_value=past_key_value,
                        output_attentions=output_attentions,
                        use_cache=use_cache,
                        cache_position=cache_position,
                    )
                    hidden_states = residual + hidden_states

                    # Fully Connected
                    residual = hidden_states
                    hidden_states = layer.post_attention_layernorm(hidden_states)
                    
                    if sequence_length > 1:
                        # print("in prefill layer ", layer_idx)
                        # 对于prefill阶段，仅将experts加载到GPU计算
                        experts = layer.block_sparse_moe.experts

                        # 将experts移动到GPU
                        if layer_idx != 0:
                            for expert in experts:
                                expert.cuda(0)

                        # 在GPU上进行MoE计算（gate保持在CPU）
                        final_hidden_states, router_logits = layer.block_sparse_moe(hidden_states)

                        # 计算完成后将experts移回CPU
                        if layer_idx != 0:
                            for expert in experts:
                                expert.w1.to('cpu')
                                expert.w2.to('cpu')
                    else:
                        # print("in decode layer", layer_idx)
                        hidden_states_flat = hidden_states.view(-1, hidden_dim)
                        ### 根据router计算需要使用的专家 ###
                        router = layer.block_sparse_moe.gate
                        # router_logits: (batch * sequence_length, n_experts)
                        router_logits = router(hidden_states_flat)

                        routing_weights = F.softmax(router_logits, dim=1, dtype=torch.float)
                        routing_weights, selected_experts = torch.topk(routing_weights, self.top_k, dim=-1)
                        routing_weights /= routing_weights.sum(dim=-1, keepdim=True)
                        # we cast back to the input dtype
                        routing_weights = routing_weights.to(hidden_states_flat.dtype)
                        expert_ids = selected_experts[0]
                        expert_weights = routing_weights[0]
                        # print('the right experts is', expert_ids)
                        ## tensor([6, 7], device='cuda:0')
                        if layer_idx > 0:
                            ### 判断加载是否正确
                            predict_experts = current_buffer.get_predict_experts()
                            
                            # 判断expert_ids和predict_experts是否包含相同数据（忽略顺序）
                            if not torch.equal(torch.sort(expert_ids)[0], torch.sort(predict_experts)[0]):
                                ### 不吻合，重新加载
                                self._load_conflict_layer(
                                    layer_idx,
                                    buffer=current_buffer,## 用cur_buffer_index
                                    expert_ids=expert_ids,
                                    hidden_states=hidden_states_flat,
                                )
                                torch.cuda.synchronize()  # 等待所有CUDA操作完成                                
                            
                            ### 使用当前缓冲区进行 MLP 计算 ###
                            final_hidden_states = current_buffer(hidden_states_flat, expert_weights, expert_ids)
                        else:
                            final_hidden_states_expert0 = layer.block_sparse_moe.experts[expert_ids[0]](
                                hidden_states_flat) * expert_weights[0]

                            final_hidden_states_expert1 = layer.block_sparse_moe.experts[expert_ids[1]](
                                hidden_states_flat) * expert_weights[1]

                            # 将两个专家的结果相加
                            final_hidden_states = final_hidden_states_expert0 + final_hidden_states_expert1

                        final_hidden_states = final_hidden_states.reshape(batch_size, sequence_length, hidden_dim)

                    hidden_states = residual + final_hidden_states

                    outputs = (hidden_states,)

                    if output_attentions:
                        outputs += (self_attn_weights,)

                    if use_cache:
                        outputs += (present_key_value,)

                    return outputs

            # 替换forward方法
            layer.forward = new_forward

def convert_mixtral_to_cached_mlp(llm, dtype, sparsity=0.9):
    ### 其他部分存放在GPU上
    llm.model.embed_tokens.cuda(0)
    for i in range(len(llm.model.layers)):
        llm.model.layers[i].self_attn.cuda(0)
        llm.model.layers[i].input_layernorm.cuda(0)
        llm.model.layers[i].post_attention_layernorm.cuda(0)
        llm.model.layers[i].block_sparse_moe.gate.cuda(0)
        for j in range(len(llm.model.layers[0].block_sparse_moe.experts)):
            llm.model.layers[i].block_sparse_moe.experts[j].w3.cuda(0)
    ### 第0层的专家存放在GPU上
    for j in range(len(llm.model.layers[0].block_sparse_moe.experts)):
        llm.model.layers[0].block_sparse_moe.experts[j].cuda(0)

    llm.model.norm.cuda(0)
    llm.lm_head.cuda(0)
    
    # 创建两个共享的CachedMLP实例
    buffer0 = CachedMLP(
        input_dim=llm.config.hidden_size,
        hidden_dim=llm.config.intermediate_size,
        dtype=dtype,
        sparsity=sparsity
    )
    buffer1 = CachedMLP(
        input_dim=llm.config.hidden_size,
        hidden_dim=llm.config.intermediate_size,
        dtype=dtype,
        sparsity=sparsity
    )
    cached_mlps = [buffer0, buffer1]
    
    for i, layer in enumerate(llm.model.layers):
        if i==0:
            continue
        # 将专家的forward方法替换为PipelineLLM管理的方式
        for j, expert in enumerate(layer.block_sparse_moe.experts):
            expert.cpu_mlp = {
                "w1": expert.w1.cpu().weight,
                "w2": expert.w2.cpu().weight.T.contiguous(),
                "w3": expert.w3,
            }
    return llm, cached_mlps

In [33]:
llm, cached_mlps = convert_mixtral_to_cached_mlp(llm, dtype, sparsity=0.1)

# 创建流水线模型
PLLM = PipelineLLM(llm, cached_mlps)

active neural num  12902
active neural num  12902


### 测试时间开销

In [34]:
import json
from datasets import load_dataset, Dataset
from transformers import GenerationConfig


input_length = 1
MAX_LENGTH = input_length
output_length = 8
test_samples = 2
device_id = 0

with open("../path.json", "r") as f:
    paths = json.load(f)
    fineweb_path = paths["fineweb"]

def preprocess_data(data, tokenizer):
	# 使用 tokenizer 将文本数据转换为模型输入
	inputs = tokenizer(data, padding="max_length", truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
	inputs["labels"] = inputs.input_ids.clone()
	return inputs

filt_type = fineweb_path.split('.')[-1]
fineweb = load_dataset(filt_type, data_files=fineweb_path) #726000
fineweb_text = fineweb['train']['text']

# 预热（避免第一次运行时的额外开销）
# for text in fineweb_text[:5] :
#     inputs = preprocess_data(text, tokenizer)
#     with torch.no_grad():
#         output = llm(input_ids=inputs["input_ids"].cuda(device_id), attention_mask=inputs["attention_mask"].cuda(device_id))

generated_all = 0
prefill_time, decode_time = 0, 0
print("output length is {}".format(output_length))
for text in fineweb_text[:test_samples] :
    inputs = preprocess_data(text, tokenizer)

    # 预热（避免第一次运行时的额外开销）
    with torch.no_grad():
        output = llm(input_ids=inputs["input_ids"].cuda(device_id), attention_mask=inputs["attention_mask"].cuda(device_id))

    # 测试时间
    start_event = torch.cuda.Event(enable_timing=True)
    end_event = torch.cuda.Event(enable_timing=True)

    # 开始计时
    torch.cuda.synchronize()
    start_event.record()

    # 前向传播
    with torch.no_grad():
        output = llm.generate(
            input_ids=inputs["input_ids"].cuda(device_id),
            attention_mask=inputs["attention_mask"].cuda(device_id),
            max_length=input_length + output_length,  # 总长度为输入长度 + 输出长度
            generation_config=GenerationConfig(do_sample=False),
            pad_token_id=tokenizer.pad_token_id, 
            # cache_implementation="static" ## moe not support
        )

    # 结束计时
    end_event.record()
    torch.cuda.synchronize()

    # 计算时间
    elapsed_time = start_event.elapsed_time(end_event) / 1000  # 转换为秒
    decode_time += elapsed_time
    print(f"Generated output length: {len(output[0]) - input_length}", f"Time taken: {elapsed_time:.4f} seconds")
    # print(output)
    print(tokenizer.batch_decode(output, skip_special_tokens=True))

    generated_all += len(output[0]) - input_length

timepertoken = (decode_time) / (generated_all)
# print("decode time:", '{:.4f}'.format((decode_time) /test_samples), ' s')
print("decode phase speed:", '{:.4f}'.format(1/timepertoken) , ' token/s')
print("the number of experts per token:", PLLM.get_reload_experts() / generated_all)

PLLM.print_reload_experts()

output length is 32
predict_experts in layer 1: ftensor([3, 5], device='cuda:0')
reloading 1 experts, 4
predict_experts in layer 5: ftensor([2, 5], device='cuda:0')
reloading 1 experts, 0
predict_experts in layer 6: ftensor([2, 4], device='cuda:0')
reloading 1 experts, 0
predict_experts in layer 8: ftensor([4, 7], device='cuda:0')
reloading 2 experts, tensor([5, 6], device='cuda:0')
predict_experts in layer 9: ftensor([7, 1], device='cuda:0')
reloading 1 experts, 2
predict_experts in layer 10: ftensor([2, 4], device='cuda:0')
reloading 1 experts, 6
predict_experts in layer 11: ftensor([5, 0], device='cuda:0')
reloading 1 experts, 6
predict_experts in layer 12: ftensor([0, 4], device='cuda:0')
reloading 1 experts, 5
predict_experts in layer 15: ftensor([7, 1], device='cuda:0')
reloading 1 experts, 3
predict_experts in layer 16: ftensor([0, 3], device='cuda:0')
reloading 1 experts, 1
predict_experts in layer 17: ftensor([0, 7], device='cuda:0')
reloading 1 experts, 6
predict_experts in l

In [6]:

########################################################################
#Save gemlite cache
if(backend == 'gemlite'):
	gemlite.core.GemLiteLinear.cache_config('/tmp/gemlite_config.json') 

#### torch.profile

trace-offloading-r.json是最优，就是做完一个index就传一个

In [23]:
import json
from datasets import load_dataset, Dataset
from transformers import GenerationConfig

input_length = 1
MAX_LENGTH = input_length
output_length = 2
test_samples = 4

with open("../path.json", "r") as f:
    paths = json.load(f)
    fineweb_path = paths["fineweb"]

def preprocess_data(data, tokenizer):
	# 使用 tokenizer 将文本数据转换为模型输入
	inputs = tokenizer(data, padding="max_length", truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
	inputs["labels"] = inputs.input_ids.clone()
	return inputs

fineweb = load_dataset("parquet",data_files=fineweb_path) #726000
fineweb_text = fineweb['train']['text'][:test_samples] 

print("output length is {}".format(output_length))
text = fineweb_text[0]
inputs = preprocess_data(text, tokenizer)

# cached_mlp.clear_load_from_cpu_stats()
with torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA,
    ]
) as p:
    # 前向传播
    with torch.no_grad():
        output = llm.generate(
            input_ids=inputs["input_ids"].cuda(),
            attention_mask=inputs["attention_mask"].cuda(),
            max_length=input_length + output_length,  # 总长度为输入长度 + 输出长度
            generation_config=GenerationConfig(do_sample=False),
            pad_token_id=tokenizer.eos_token_id
        )
print(p.key_averages().table(
    sort_by="self_cpu_time_total", row_limit=-1))
p.export_chrome_trace("./offloading-hqq2-reload.json")

output length is 2
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                            aten::index        33.20%     271.076ms        33.65%     274.725ms     358.181us       1.808ms         0.33%       1.808ms       2.357us           767  
                                  cudaDeviceSynchronize        19.75%     161.244ms        19.75%     161.244ms       2.687ms      51.745us         0.01%      51.745us       0.862us       

#### 测试一个正常输出

In [35]:
import json
from datasets import load_dataset, Dataset
from transformers import GenerationConfig

input_length = 6
MAX_LENGTH = input_length
output_length = 6
device_id = 0
test_samples = 1

def preprocess_data(data, tokenizer):
	# 使用 tokenizer 将文本数据转换为模型输入
	inputs = tokenizer(data, padding="max_length", truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
	inputs["labels"] = inputs.input_ids.clone()
	return inputs

generated_all, decode_time = 0, 0
print("max output length is {}".format(output_length))
text = "The future of AI is "
inputs = preprocess_data(text, tokenizer)
# 测试时间
start_event = torch.cuda.Event(enable_timing=True)
end_event = torch.cuda.Event(enable_timing=True)

# 开始计时
torch.cuda.synchronize()
start_event.record()

# 前向传播
with torch.no_grad():
    output = llm.generate(
        input_ids=inputs["input_ids"].cuda(device_id),
        attention_mask=inputs["attention_mask"].cuda(device_id),
        max_length=input_length + output_length,  # 总长度为输入长度 + 输出长度
        generation_config=GenerationConfig(do_sample=False),
        pad_token_id=tokenizer.pad_token_id, 
        # cache_implementation="static" ## moe not support
    )

# 结束计时
end_event.record()
torch.cuda.synchronize()

# 计算时间
elapsed_time = start_event.elapsed_time(end_event) / 1000  # 转换为秒
decode_time += elapsed_time
print(f"Generated output length: {len(output[0]) - input_length}", f"Time taken: {elapsed_time:.4f} seconds")
# print(output)
print(tokenizer.batch_decode(output, skip_special_tokens=True))

generated_all += len(output[0]) - input_length

timepertoken = (decode_time) / (generated_all)
print("decode phase speed:", '{:.4f}'.format(1/timepertoken) , ' token/s')
print("the number of experts per token:", PLLM.get_reload_experts() / generated_all)

max output length is 6


predict_experts in layer 1: ftensor([7, 6], device='cuda:0')
reloading 2 experts, tensor([4, 2], device='cuda:0')
predict_experts in layer 2: ftensor([2, 1], device='cuda:0')
reloading 2 experts, tensor([6, 3], device='cuda:0')
predict_experts in layer 3: ftensor([7, 2], device='cuda:0')
reloading 1 experts, 1
predict_experts in layer 5: ftensor([3, 5], device='cuda:0')
reloading 1 experts, 7
predict_experts in layer 6: ftensor([4, 0], device='cuda:0')
reloading 1 experts, 5
predict_experts in layer 8: ftensor([1, 4], device='cuda:0')
reloading 1 experts, 2
predict_experts in layer 9: ftensor([7, 1], device='cuda:0')
reloading 1 experts, 6
predict_experts in layer 14: ftensor([2, 0], device='cuda:0')
reloading 1 experts, 6
predict_experts in layer 15: ftensor([2, 3], device='cuda:0')
reloading 1 experts, 4
predict_experts in layer 16: ftensor([0, 2], device='cuda:0')
reloading 1 experts, 3
predict_experts in layer 17: ftensor([3, 0], device='cuda:0')
reloading 1 experts, 1
predict_expe

### 加载到GPU上

In [1]:
from transformers import MixtralForCausalLM, AutoTokenizer
import torch
import torch.nn as nn
from typing import Optional
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"
import json

def get_model(model_name, device_map, dtype=torch.bfloat16):
    llm = MixtralForCausalLM.from_pretrained(
        model_name,
        device_map=device_map,
        use_cache=True,
        torch_dtype=dtype,
    ) 
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

    return llm, tokenizer

with open('../path.json', 'r') as f:
    path = json.load(f)
    model_name = path['mixtral']
    # threshold_path = path[threshold_path_name]

with open('../quantize/device_map_1.json', 'r') as f:
    device_map = json.load(f)

dtype = torch.float16
llm, tokenizer = get_model(model_name, device_map, dtype=dtype)

/home/bcds/.conda/envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 19/19 [00:25<00:00,  1.36s/it]
